In [1]:
from transformers import AutoModelForCausalLM, OPTForCausalLM, AutoTokenizer,GenerationConfig
from peft import get_peft_config, get_peft_model
from helper.prompter import Prompter
import torch
import gc
import os

In [1]:
import os 

import pandas as pd
directory = "/home/natdanai/LLaMA-Factory/saves/E3"
#want merge csv
for filename in os.listdir(directory):
    print(filename)
    

openthaigpt_funirture_kwgpt4_train_augment
typhoon_cosmetic_kwgpt4_train_augment
openthaigpt_cosmetic_kwgpt4_train_augment
typhoon_funirture_kwgpt4_train_augment


In [16]:
import pandas as pd
import random
df=pd.read_csv("/home/natdanai/Download/dataset/train/funirture_kwgpt4_train.csv")
new_data=[]
for round in range(2):
    for i in range(len(df)):
        list_input=df['input'][i].split(",")
        random.shuffle(list_input)
        list_output=df['output'][i]
        new_data.append({"input":",".join(list_input),"output":list_output,"instruction":"สร้างประโยคโฆษณาโดยใช้คำที่กำหนดให้"})
dfnew=pd.DataFrame(new_data)
dfnew=pd.concat([df,dfnew])
dfnew.to_csv("/home/natdanai/Download/dataset/train/funirture_kwgpt4_train_shuffle.csv",index=False)
print(len(new_data))
for round in range(2):
    for i in range(len(df)):
        list_input=df['input'][i].split(",")
        random.shuffle(list_input)
        if len(list_input) <= 4:
            min_len = 1
            max_len = len(list_input)
        elif len(list_input) <= 8:
            min_len = 2
            max_len = len(list_input)//2
        else:
            min_len = len(list_input)//3
            max_len = len(list_input)//2
        # print(min_len, max_len)
        num_to_keep = random.randint(min_len,max_len)
        text = ','.join(list_input[:num_to_keep])
        list_output=df['output'][i]
        new_data.append({"input":text,"output":list_output,"instruction":"สร้างประโยคโฆษณาโดยใช้คำที่กำหนดให้"})
print(len(new_data))
dfnew=pd.concat([df,pd.DataFrame(new_data)])
dfnew.to_csv("/home/natdanai/Download/dataset/train/funirture_kwgpt4_train_shuffle_augment.csv",index=False)


20228
40456


In [17]:
import pandas as pd
import random
df=pd.read_csv("/home/natdanai/Download/dataset/train/cosmetic_kwgpt4_train.csv")
new_data=[]
for round in range(2):
    for i in range(len(df)):
        list_input=df['input'][i].split(",")
        random.shuffle(list_input)
        list_output=df['output'][i]
        new_data.append({"input":",".join(list_input),"output":list_output,"instruction":"สร้างประโยคโฆษณาโดยใช้คำที่กำหนดให้"})
dfnew=pd.DataFrame(new_data)
dfnew=pd.concat([df,dfnew])
dfnew.to_csv("/home/natdanai/Download/dataset/train/cosmetic_kwgpt4_train_shuffle.csv",index=False)
print(len(new_data))
for round in range(2):
    for i in range(len(df)):
        list_input=df['input'][i].split(",")
        random.shuffle(list_input)
        if len(list_input) <= 4:
            min_len = 1
            max_len = len(list_input)
        elif len(list_input) <= 8:
            min_len = 2
            max_len = len(list_input)//2
        else:
            min_len = len(list_input)//3
            max_len = len(list_input)//2
        # print(min_len, max_len)
        num_to_keep = random.randint(min_len,max_len)
        text = ','.join(list_input[:num_to_keep])
        list_output=df['output'][i]
        new_data.append({"input":text,"output":list_output,"instruction":"สร้างประโยคโฆษณาโดยใช้คำที่กำหนดให้"})
print(len(new_data))
dfnew=pd.concat([df,pd.DataFrame(new_data)])
dfnew.to_csv("/home/natdanai/Download/dataset/train/cosmetic_kwgpt4_train_shuffle_augment.csv",index=False)


17206
34412


In [8]:
import pandas as pd
import random
df=pd.read_csv('/home/natdanai/Download/dataset/test/cosmetic_kwgpt4_test.csv')
new_data=[]
for index, row in df.iterrows():
    words = row['input'].split(',')
    for i in range(2):
        random.shuffle(words)

        if len(words) < 4:
            min_len = 1
            max_len = len(words)
        elif len(words) <= 8:
            min_len = 2
            max_len = len(words)//2
        else:
            min_len = 3
            max_len = len(words)//2
        # print(min_len, max_len)
        num_to_keep = random.randint(min_len,max_len)
        text = ','.join(words[:num_to_keep])
        new_dict = {'input': text, 'output': row['output'], 'instruction': row['instruction']}
        new_data.append(new_dict)
dfnew = pd.DataFrame(new_data)
dfnew.to_csv("/home/natdanai/Download/dataset/test/cosmetic_kwgpt4_test_augment.csv",index=False)

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import pandas as pd
import evaluate
from evaluate import logging
import numpy as np
from torch.nn import CrossEntropyLoss
import gc
model_id="openthaigpt/openthaigpt-1.0.0-beta-13b-chat-hf"
model = AutoModelForCausalLM.from_pretrained(
                model_id,
                # low_cpu_mem_usage=True,
                device_map="auto",
                torch_dtype=torch.bfloat16,
            )

batch_size=1
add_start_token= True
max_length=None
tokenizer = AutoTokenizer.from_pretrained(model_id)
device ="cuda"
# path_name="output_E4_openthaigpt_cosmetic_gpt4_shuffle_augment_test_augment"
# path_name="output_E4_openthaigpt_cosmetic_gpt4_shuffle_augment_test"
# path_name="output_E4_openthaigpt_cosmetic_gpt4_shuffle_test_augment"
import glob
import os
#find path with have "openthaigpt" in name
path_load="/home/natdanai/KeyToad/script/gentest"
entries = os.listdir(path_load)
sorted_entries = sorted(entries)
list_load=[]
for file in sorted_entries:
    if "openthaigpt" in file:
        list_load.append(os.path.join(path_load, file))
perplexity_list=[]
file_list=[]
for path in list_load:
    print(path)
    path_name=path.split("/")[-1].split(".")[0]
    df=pd.read_csv(f"gentest/{path_name}.csv")
    predictions=df['response'].tolist()
    # if batch_size > 1 (which generally leads to padding being required), and
    # if there is not an already assigned pad_token, assign an existing
    # special token to also be the padding token
    if tokenizer.pad_token is None and batch_size > 1:
        existing_special_tokens = list(tokenizer.special_tokens_map_extended.values())
        # check that the model already has at least one special token defined
        assert (
            len(existing_special_tokens) > 0
        ), "If batch_size > 1, model must have at least one special token to use for padding. Please use a different model or set batch_size=1."
        # assign one of the special tokens to also be the pad token
        tokenizer.add_special_tokens({"pad_token": existing_special_tokens[0]})

    if add_start_token and max_length:
        # leave room for <BOS> token to be added:
        assert (
            tokenizer.bos_token is not None
        ), "Input model must already have a BOS token if using add_start_token=True. Please use a different model, or set add_start_token=False"
        max_tokenized_len = max_length - 1
    else:
        max_tokenized_len = max_length
    tokenizer.pad_token = tokenizer.eos_token
    encodings = tokenizer(
        predictions,
        add_special_tokens=False,
        padding=True,
        truncation=True if max_tokenized_len else False,
        max_length=max_tokenized_len,
        return_tensors="pt",
        return_attention_mask=True,
        
    ).to(device)

    encoded_texts = encodings["input_ids"]
    attn_masks = encodings["attention_mask"]

    # check that each input is long enough:
    if add_start_token:
        assert torch.all(torch.ge(attn_masks.sum(1), 1)), "Each input text must be at least one token long."
    else:
        assert torch.all(
            torch.ge(attn_masks.sum(1), 2)
        ), "When add_start_token=False, each input text must be at least two tokens long. Run with add_start_token=True if inputting strings of only one token, and remove all empty input strings."

    ppls = []
    loss_fct = CrossEntropyLoss(reduction="none")

    for start_index in logging.tqdm(range(0, len(encoded_texts), batch_size)):
        end_index = min(start_index + batch_size, len(encoded_texts))
        encoded_batch = encoded_texts[start_index:end_index]
        attn_mask = attn_masks[start_index:end_index]

        if add_start_token:
            bos_tokens_tensor = torch.tensor([[tokenizer.bos_token_id]] * encoded_batch.size(dim=0)).to(device)
            encoded_batch = torch.cat([bos_tokens_tensor, encoded_batch], dim=1)
            attn_mask = torch.cat(
                [torch.ones(bos_tokens_tensor.size(), dtype=torch.int64).to(device), attn_mask], dim=1
            )

        labels = encoded_batch

        with torch.no_grad():
            out_logits = model(encoded_batch, attention_mask=attn_mask).logits

        shift_logits = out_logits[..., :-1, :].contiguous()
        shift_labels = labels[..., 1:].contiguous()
        shift_attention_mask_batch = attn_mask[..., 1:].contiguous()

        perplexity_batch = torch.exp(
            (loss_fct(shift_logits.transpose(1, 2), shift_labels) * shift_attention_mask_batch).sum(1)
            / shift_attention_mask_batch.sum(1)
        )

        ppls += perplexity_batch.tolist()
    print(f"Perplexity: {np.mean(ppls)}")
    perplexity_list.append(np.mean(ppls))
    file_list.append(path_name)
    del encodings,encoded_texts,attn_masks,ppls
    gc.collect()
    torch.cuda.empty_cache()
    print("end")
df=pd.DataFrame({'file':file_list,'perplexity':perplexity_list})
df.to_csv("perplexity_openthai.csv",index=False)



2024-02-20 18:39:11.903206: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-20 18:39:11.903283: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-20 18:39:11.904286: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-20 18:39:11.911130: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-20 18:39:13.009252: W tensorflow/compiler/tf2

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/home/natdanai/miniconda3/envs/ktd/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


/home/natdanai/KeyToad/script/gentest/output_E4_openthaigpt_cosmetic_gpt4_shuffle_augment_test.csv


  0%|          | 0/956 [00:00<?, ?it/s]

...

Perplexity: 9333.720418386612
end
/home/natdanai/KeyToad/script/gentest/output_E5_openthaigpt_furniture_gpt4_shuffle_test.csv


  0%|          | 0/1124 [00:00<?, ?it/s]

Perplexity: 60933.99206691111
end
/home/natdanai/KeyToad/script/gentest/output_E5_openthaigpt_furniture_gpt4_shuffle_test_augment.csv


  0%|          | 0/2248 [00:00<?, ?it/s]

Perplexity: 2982676096.374689
end


In [14]:
import pandas as pd
from transformers import AutoTokenizer

# Assuming you've already loaded the dataset into a DataFrame
df = pd.read_csv('/home/natdanai/Download/dataset/test/funirture_kwgpt4_test.csv')

# Load the tokenizer
model_id = "scb10x/typhoon-7b"
tokenizer = AutoTokenizer.from_pretrained(model_id)

while True:
    random_rows = df.sample(5)
    max_list=[]
    input_list=[]
    outputbatch=[]
    for index, row in random_rows.iterrows():
        tokens = tokenizer(row['output'], return_tensors='pt')['input_ids']
        num_tokens = tokens.size(1)  # Number of tokens in the row
        max_list.append(num_tokens)
        input_list.append(row['input'])
        outputbatch.append(row['output'])
    if max(max_list)<70 and min(max_list)>30:
        break
print(input_list)
print(outputbatch)
print(max_list)
df=pd.DataFrame({'input':input_list,'output':outputbatch})
df.to_csv('test5funirutre.csv',index=False)


['ความหอมหวาน,เทียนหอม,บรรยากาศเงียบสงบ,กลิ่นดิน,เทียน,ป่าและกองไฟ,กลิ่นของบ้าน', 'แผ่นไม้,แผ่นกระจก,ตู้,สิ่งของ,คลายจากกระแทก,รับน้ำหนัก,ชั้นวางของ,ปุ่มรับชั้น,ยึดติด,ชั้น,กระจก,ตู้โชว์', 'โครงตู้,ช่องว่าง,ตู้,ล็อก,แผ่นปิดตู้เข้ามุม,เดือย,บานตู้,ตู้เข้ามุม,ตู้ตั้งพื้น', 'โครงตู้,ลวดตาข่าย,ตู้,อุปกรณ์อิเลคทรอนิก,บานตู้,รีโมทคอนโทรล', 'มียางยืดรัด,ปูที่นอน,ดูแลรักษาง่าย,ผ้ายืดได้,ผ้า,ผ้าเจอร์ซีย์ถัก']
['กลิ่นของบ้านคุณเป็นอย่างไร เทียนหอม ให้กลิ่นดินและกองไฟ แฝงความหอมหวาน ยินดีต้อนรับกลับบ้าน สู่บรรยากาศเงียบสงบซึ่งได้รับแรงบันดาลใจจากป่าและกองไฟที่แสนอบอุ่น', 'ปุ่มรับชั้น สำหรับใช้งานกับแผ่นกระจก หรือแผ่นไม้บาง เพื่อใช้วางสิ่งของที่มีน้ำหนักไม่มากในตู้โชว์ และชั้นวางของแบบฉบับ โดยต้องยึดติดกับปุ่มรับชั้นตัวเมียให้แน่น และตรวจสอบอีกครั้งว่าขันแน่นแล้ว ป้องกันการคลายจากกระแทก หรือการรับน้ำหนัก', 'ด้วยแผ่นปิดตู้เข้ามุม ช่วยให้คุณเนรมิตตู้ตั้งพื้นให้กลายเป็นตู้เข้ามุม และเพิ่มบานตู้  เพื่อปิดช่องว่างมีเดือยที่ทำให้ประกอบง่าย แค่คลิกให้ลงล็อกก็ประกอบชิ้นส่วนต่าง ๆ เข้าด้วยกันได้', 'บานตู้เ

In [7]:
del model
del inputs
del output
gc.collect()
torch.cuda.empty_cache()

In [8]:
model_id="scb10x/typhoon-7b"
prompt="mistral"
prompter_gen=Prompter(prompt)
tokenizer = AutoTokenizer.from_pretrained(model_id)
context = prompter_gen.generate_prompt("สร้างประโยคโฆษณาโดยใช้คำที่กำหนดให้","ลิปสติก")



model = AutoModelForCausalLM.from_pretrained(
                model_id,
                low_cpu_mem_usage=True,
                device_map="auto",
                torch_dtype=torch.bfloat16,
            )
model.load_adapter("/home/natdanai/KeyToad/script/model/E3/typhoon_E3_shuffle_augment_gpt4")
model.load_adapter("/home/natdanai/KeyToad/script/model/E3/typhoon_E3_shuffle_gpt4")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
generation_config = GenerationConfig(
            temperature=0.9,
            top_p=0.96,
            top_k=50,
            do_sample=True,
            typical_p=1.0,
        )
max_length=100
inputs = tokenizer(context, return_tensors="pt").to("cuda")
print("enable_adapters")
model.enable_adapters()
output = model.generate(
                input_ids=inputs["input_ids"],
                max_new_tokens=max_length, # 512
                no_repeat_ngram_size=4,
                generation_config=generation_config,
                pad_token_id=tokenizer.eos_token_id,
                )

print(tokenizer.decode(output[0], skip_special_tokens=True))
print("disable_adapters")
model.disable_adapters()
output = model.generate(
                input_ids=inputs["input_ids"],
                max_new_tokens=max_length, # 512
                no_repeat_ngram_size=4,
                generation_config=generation_config,
                pad_token_id=tokenizer.eos_token_id,
                )
print(tokenizer.decode(output[0], skip_special_tokens=True))

enable_adapters
[INST] สร้างประโยคโฆษณาโดยใช้คำที่กำหนดให้ ลิปสติก [/INST]ลิปสติกเนื้อแมทมอบผลลัพธ์เรียบลื่นดุจผ้าซาติน ด้วยการปาดเพียงครั้งเดียวแต่เปี่ยมไปด้วยสีสัน และความมีชีวิตชีวา จากเม็ดสีเข้มข้น ให้ความรู้สึกเบาสบาย และติดทนนานด้วยเทคโนโลยี ให้ริมฝีปากดูเรียบเนียนราวกับผ่านการแต่งหน้ามาอย่างดีเยี่ยม และทำให้เรียวปากดูอวบอิ่มตลอดวัน รุ่นพิเศษ ริมฝีปากสีชมพูระเร
disable_adapters
[INST] สร้างประโยคโฆษณาโดยใช้คำที่กำหนดให้ ลิปสติก [/INST]
คำโปรย
สร้างประโยคโฆษณาด้วยการใส่คำที่กำหนดไว้ ให้กับลิปสติก
รายละเอียด
[INST] สร้างสรรค์ประโยคโฆษณาของคุณด้วยการใส่คำที่คุณกำหนดไว้บนลิปสตรัคเตอร์ของเรา ให้กับลิสสติก! โดยคุณสามารถสร้างประโยคได้มากมาย และคุณจะสามารถเพิ่มหรือลดข้อความลงไปได้ตามที่คุณต้องการ!
[INST 2] ลิปสตรัสเตอร์ของคุณจะเป็นตัวละครที่มีชีวิต! ลิปสตรา


In [6]:
generation_config = GenerationConfig(
            temperature=0.9,
            top_p=0.96,
            top_k=50,
            do_sample=True,
            typical_p=1.0,
        )
max_length=100
inputs = tokenizer(context, return_tensors="pt").to("cuda")
print("enable_adapters")
model.enable_adapters()
output = model.generate(
                input_ids=inputs["input_ids"],
                max_new_tokens=max_length, # 512
                no_repeat_ngram_size=4,
                generation_config=generation_config,
                pad_token_id=tokenizer.eos_token_id,
                )

print(tokenizer.decode(output[0], skip_special_tokens=True))
print("disable_adapters")
model.disable_adapters()
output = model.generate(
                input_ids=inputs["input_ids"],
                max_new_tokens=max_length, # 512
                no_repeat_ngram_size=4,
                generation_config=generation_config,
                pad_token_id=tokenizer.eos_token_id,
                )
print(tokenizer.decode(output[0], skip_special_tokens=True))

enable_adapters
[INST] สร้างประโยคโฆษณาโดยใช้คำที่กำหนดให้ ลิปสติก [/INST]ลิปสติก เป็นลิปสเตนเนื้อแมท สูตรติดทน ใหม่ล่าสุด ไม่หลุดเป็นขุย แม้จะทาเพียงครั้งแรก ผสานคุณค่าการบำรุงจากน้ำมันอัลมอนด์ และเชียบัตเตอร์ ให้ริมฝีปากรู้สึกชุ่มชื้นตลอดวัน เกลี่ยง่าย เบาสบายริมฝีปาก โดยไม่รู้สึกแห้งหรือหนัก มาในเฉดสีใหม่ กว่าเดิม เริ่มจากเฉดอ่อนอย่างอ่อนหวาน ไปจนถึงเฉดสีสดใสสะด
disable_adapters
[INST] สร้างประโยคโฆษณาโดยใช้คำที่กำหนดให้ ลิปสติก [/INST]
หากคุณต้องการสร้างโฆษณาที่กำหนดให้ใช้เฉพาะคำ คุณจะสามารถสร้างโฆษณากับเราเพื่อกำหนดตัวเลือกของคุณได้
ดังนั้นคุณจะต้องติดต่อเราและอธิบายความต้องการของคุณ
เราสามารถจัดหาคำที่จำเป็นสำหรับการโฆษณาบน Google AdWords, Bing Ads, และโฆษณาผ่านอีเมลได้
สำหรับบทความนี้ เราจะแสดงตัวอย่างจากโฆษณาที่มีการใช้คำในลักษณะนี้
เช่น คุณสามารถต้องการโฆษณาที่มีข้อจำกัดด้านคำเพียงสามคำ ดังนี้
“


In [1]:
import os
import glob
import pandas as pd 
import evaluate
directories_starting_with_E = glob.glob("/home/natdanai/KeyToad/script/gentest")
bleu = evaluate.load("bleu")
path_E="/home/natdanai/KeyToad/script/gentest/"
entries = os.listdir(path_E)
sorted_entries = sorted(entries)
dataframe_list=[]
for filename in sorted_entries:
    print("*"*10)
    print(path_E)
    print(filename)
    df=pd.read_csv(path_E+filename)
    # references=df['output'].tolist()
    predictions=df['response'].tolist()
    if "cosmetic" in filename:
        if "test_augment" in filename:
            references=pd.read_csv("/home/natdanai/Download/dataset/test/cosmetic_kwgpt4_test_augment.csv")['output'].tolist()
        else:
            references=pd.read_csv("/home/natdanai/Download/dataset/test/cosmetic_kwgpt4_test.csv")['output'].tolist()
    elif "furniture" in filename:
        if "test_augment" in filename:
            references=pd.read_csv("/home/natdanai/Download/dataset/test/funirture_kwgpt4_test_augment.csv")['output'].tolist()
        else:
            references=pd.read_csv("/home/natdanai/Download/dataset/test/funirture_kwgpt4_test.csv")['output'].tolist()
    dict_result={}
    dict_result['filename']=filename
    result_bleurt=bleu.compute(predictions=predictions, references=references)
    dict_result['bleu']=result_bleurt['bleu']
    dataframe_list.append(dict_result)
    print("*"*10)
df=pd.DataFrame(dataframe_list)
df.to_csv(f"result_bleu.csv",index=False)


2024-02-22 15:14:30.683438: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-22 15:14:30.683519: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-22 15:14:30.684688: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-22 15:14:30.702812: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-22 15:14:32.672729: W tensorflow/compiler/tf2

**********
/home/natdanai/KeyToad/script/gentest/
output_E0_wangchanglm_cosmetic_gpt4_test.csv
**********
**********
/home/natdanai/KeyToad/script/gentest/
output_E0_wangchanglm_cosmetic_noinput_gpt4_test.csv
**********
**********
/home/natdanai/KeyToad/script/gentest/
output_E1_typhoon_cosmetic_kwgpt4_test.csv
**********
**********
/home/natdanai/KeyToad/script/gentest/
output_E1_typhoon_cosmetic_kwpostag_test.csv
**********
**********
/home/natdanai/KeyToad/script/gentest/
output_E2_typhoon_cosmetic_kwgpt4_neftune_test.csv
**********
**********
/home/natdanai/KeyToad/script/gentest/
output_E2_typhoon_cosmetic_kwpostag_neftune_test.csv
**********
**********
/home/natdanai/KeyToad/script/gentest/
output_E3_typhoon_cosmetic_gpt4_shuffle_augment_test.csv
**********
**********
/home/natdanai/KeyToad/script/gentest/
output_E3_typhoon_cosmetic_gpt4_shuffle_augment_test_augment.csv
**********
**********
/home/natdanai/KeyToad/script/gentest/
output_E3_typhoon_cosmetic_gpt4_shuffle_test.csv
*

In [2]:
import os
import glob
import pandas as pd 
import evaluate
directories_starting_with_E = glob.glob("/home/natdanai/KeyToad/script/gentest")
bleurt = evaluate.load("bleurt", module_type="metric",config_name ="BLEURT-20")
path_E="/home/natdanai/KeyToad/script/gentest/"
entries = os.listdir(path_E)
sorted_entries = sorted(entries)
dataframe_list=[]
for filename in sorted_entries:
    print("*"*10)
    print(path_E)
    print(filename)
    df=pd.read_csv(path_E+filename)
    # references=df['output'].tolist()
    predictions=df['response'].tolist()
    if "cosmetic" in filename:
        if "test_augment" in filename:
            references=pd.read_csv("/home/natdanai/Download/dataset/test/cosmetic_kwgpt4_test_augment.csv")['output'].tolist()
        else:
            references=pd.read_csv("/home/natdanai/Download/dataset/test/cosmetic_kwgpt4_test.csv")['output'].tolist()
    elif "furniture" in filename:
        if "test_augment" in filename:
            references=pd.read_csv("/home/natdanai/Download/dataset/test/funirture_kwgpt4_test_augment.csv")['output'].tolist()
        else:
            references=pd.read_csv("/home/natdanai/Download/dataset/test/funirture_kwgpt4_test.csv")['output'].tolist()
    dict_result={}
    dict_result['filename']=filename
    result_bleurt=bleurt.compute(predictions=predictions, references=references)
    dict_result['bleurt']=result_bleurt['scores']
    dataframe_list.append(dict_result)
    print("*"*10)
df=pd.DataFrame(dataframe_list)
df.to_csv(f"result_bleurt.csv",index=False)


2024-02-20 15:56:34.842654: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-20 15:56:34.842718: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-20 15:56:34.843689: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-20 15:56:34.850647: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-20 15:56:36.049760: W tensorflow/compiler/tf2

INFO:tensorflow:Reading checkpoint /home/natdanai/.cache/huggingface/metrics/bleurt/BLEURT-20/downloads/extracted/5ebfa9a61899d1c84faf91857420d22ba7a618034f74f59a83e3d33e42079f23/BLEURT-20.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint BLEURT-20
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:BLEURT-20
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... max_seq_length:512
INFO:tensorflow:... vocab_file:None
INFO:tensorflow:... do_lower_case:None
INFO:tensorflow:... sp_model:sent_piece
INFO:tensorflow:... dynamic_seq_length:True
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating SentencePiece tokenizer.
INFO:tensorflow:Creating SentencePiece tokenizer.
INFO:tensorflow:Will load model: /home/natdanai/.cache/huggingface/metrics/bleurt/BLEURT-20/downloads/extracted/5ebfa9a61899d1c84faf91857420d22ba7a618034f74f59a83e3d33e42079f23/BLEURT-20/sent_piece.model.
INFO:tensorflow:Sentence

2024-02-20 15:56:43.933147: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38380 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:90:00.0, compute capability: 8.0


INFO:tensorflow:BLEURT initialized.


INFO:tensorflow:BLEURT initialized.


**********
/home/natdanai/KeyToad/script/gentest/
output_E0_wangchanglm_cosmetic_gpt4_test.csv
**********
**********
/home/natdanai/KeyToad/script/gentest/
output_E0_wangchanglm_cosmetic_noinput_gpt4_test.csv
**********
**********
/home/natdanai/KeyToad/script/gentest/
output_E1_typhoon_cosmetic_kwgpt4_test.csv
**********
**********
/home/natdanai/KeyToad/script/gentest/
output_E1_typhoon_cosmetic_kwpostag_test.csv
**********
**********
/home/natdanai/KeyToad/script/gentest/
output_E2_typhoon_cosmetic_kwgpt4_neftune_test.csv
**********
**********
/home/natdanai/KeyToad/script/gentest/
output_E2_typhoon_cosmetic_kwpostag_neftune_test.csv
**********
**********
/home/natdanai/KeyToad/script/gentest/
output_E3_typhoon_cosmetic_gpt4_shuffle_augment_test.csv
**********
**********
/home/natdanai/KeyToad/script/gentest/
output_E3_typhoon_cosmetic_gpt4_shuffle_augment_test_augment.csv
**********
**********
/home/natdanai/KeyToad/script/gentest/
output_E3_typhoon_cosmetic_gpt4_shuffle_test.csv
*

In [1]:
import os
import glob
import pandas as pd 
import evaluate
perplexity = evaluate.load("perplexity", module_type="metric")
bertscore = evaluate.load("bertscore")
rouge = evaluate.load('rouge')
google_bleu = evaluate.load("google_bleu")
path_E="/home/natdanai/KeyToad/script/gentest/"
entries = os.listdir(path_E)
sorted_entries = sorted(entries)
print(len(sorted_entries))
dataframe_list=[]
for filename in sorted_entries:
        print("*"*10)
        print(path_E)
        print(filename)
        df=pd.read_csv(path_E+filename)
        # references=df['output'].tolist()
        predictions=df['response'].tolist()
        if "cosmetic" in filename:
            if "test_augment" in filename:
                references=pd.read_csv("/home/natdanai/Download/dataset/test/cosmetic_kwgpt4_test_augment.csv")['output'].tolist()
            else:
                references=pd.read_csv("/home/natdanai/Download/dataset/test/cosmetic_kwgpt4_test.csv")['output'].tolist()
        elif "furniture" in filename:
            if "test_augment" in filename:
                references=pd.read_csv("/home/natdanai/Download/dataset/test/funirture_kwgpt4_test_augment.csv")['output'].tolist()
            else:
                references=pd.read_csv("/home/natdanai/Download/dataset/test/funirture_kwgpt4_test.csv")['output'].tolist()
        dict_result={} 
        if "typhoon" in filename:
            model_id="scb10x/typhoon-7b"
            # result_perplexity=perplexity.compute(model_id=model_id,predictions=predictions,batch_size=4)
            dict_result['mean_perplexity']=0
        elif "wang" in filename:
            model_id="pythainlp/wangchanglm-7.5B-sft-enth"
            # result_perplexity=perplexity.compute(model_id=model_id,predictions=predictions,batch_size=4)
            dict_result['mean_perplexity']=0
        elif "open" in filename:
            model_id="openthaigpt/openthaigpt-1.0.0-beta-7b-chat"
            # result_perplexity=perplexity.compute(model_id=model_id,predictions=predictions,batch_size=1)
            dict_result['mean_perplexity']=0
        else:
            print("Error")
            break
        dict_result['filename']=filename
        result_google_bleu=google_bleu.compute(predictions=predictions, references=references)
        dict_result['google_bleu']=result_google_bleu['google_bleu']
        result_rouge=rouge.compute(predictions=predictions, references=references)
        dict_result['rouge1']=result_rouge['rouge1']
        dict_result['rouge2']=result_rouge['rouge2']
        dict_result['rougeL']=result_rouge['rougeL']
        result_bertscore=bertscore.compute(predictions=predictions, references=references, lang="th")
        dict_result['bertscore_precision']=result_bertscore['precision']
        dict_result['bertscore_recall']=result_bertscore['recall']
        dict_result['bertscore_f1']=result_bertscore['f1']
        dataframe_list.append(dict_result)
        print("*"*10)
df=pd.DataFrame(dataframe_list)
df.to_csv(f"result.csv",index=False)


2024-02-20 16:31:50.567357: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-20 16:31:50.567432: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-20 16:31:50.568438: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-20 16:31:50.575775: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-20 16:31:51.744309: W tensorflow/compiler/tf2

31
**********
/home/natdanai/KeyToad/script/gentest/
output_E0_wangchanglm_cosmetic_gpt4_test.csv
**********
**********
/home/natdanai/KeyToad/script/gentest/
output_E0_wangchanglm_cosmetic_noinput_gpt4_test.csv
**********
**********
/home/natdanai/KeyToad/script/gentest/
output_E1_typhoon_cosmetic_kwgpt4_test.csv
**********
**********
/home/natdanai/KeyToad/script/gentest/
output_E1_typhoon_cosmetic_kwpostag_test.csv
**********
**********
/home/natdanai/KeyToad/script/gentest/
output_E2_typhoon_cosmetic_kwgpt4_neftune_test.csv
**********
**********
/home/natdanai/KeyToad/script/gentest/
output_E2_typhoon_cosmetic_kwpostag_neftune_test.csv
**********
**********
/home/natdanai/KeyToad/script/gentest/
output_E3_typhoon_cosmetic_gpt4_shuffle_augment_test.csv
**********
**********
/home/natdanai/KeyToad/script/gentest/
output_E3_typhoon_cosmetic_gpt4_shuffle_augment_test_augment.csv
**********
**********
/home/natdanai/KeyToad/script/gentest/
output_E3_typhoon_cosmetic_gpt4_shuffle_test.cs

In [2]:
import pandas as pd
import ast
import numpy as np
import glob
dataframe_list=[]
directories_starting_with_E = glob.glob("/home/natdanai/KeyToad/script/E*")


df = pd.read_csv(f"result.csv")
df_bleurt = pd.read_csv(f"result_bleurt.csv")

list_filename=df['filename'].tolist()
list_bertscore_precision=df['bertscore_precision'].tolist()
list_bertscore_recall=df['bertscore_recall'].tolist()
list_bertscore_f1=df['bertscore_f1'].tolist()
list_bleurt=df_bleurt['bleurt'].tolist()
list_rouge1=df['rouge1'].tolist()
list_rouge2=df['rouge2'].tolist()
list_rougeL=df['rougeL'].tolist()
list_google_bleu=df['google_bleu'].tolist()
# list_mean_perplexity=df['mean_perplexity'].tolist()
for i in range(len(list_filename)):
    dict_result={}
    dict_result['filename']=list_filename[i]
    dict_result['mean_bertscore_precision']=np.mean(ast.literal_eval(list_bertscore_precision[i]))
    dict_result['mean_bertscore_recall']=np.mean(ast.literal_eval(list_bertscore_recall[i]))
    dict_result['mean_bertscore_f1']=np.mean(ast.literal_eval(list_bertscore_f1[i]))
    dict_result['mean_bleurt']=np.mean(ast.literal_eval(list_bleurt[i]))
    dict_result['rouge1']=list_rouge1[i]
    dict_result['rouge2']=list_rouge2[i]
    dict_result['rougeL']=list_rougeL[i]
    # dict_result['mean_perplexity']=list_mean_perplexity[i]
    dict_result['google_bleu']=list_google_bleu[i]
    dataframe_list.append(dict_result)
df=pd.DataFrame(dataframe_list)
df.to_csv(f"result_compare.csv",index=False)


In [5]:
df_bleurt = pd.read_csv(f"result_bleurt.csv")
df_bleurt

,filename,bleurt
0,output_E1_typhoon_cosmetic_kwgpt4_test.csv,"[0.41528409719467163, 0.422873318195343, 0.361..."
1,output_E1_typhoon_cosmetic_kwpostag_test.csv,"[0.41318750381469727, 0.5656413435935974, 0.34..."
2,output_E2_typhoon_cosmetic_kwgpt4_neftune_test...,"[0.5268957018852234, 0.4750182628631592, 0.560..."
3,output_E2_typhoon_cosmetic_kwpostag_neftune_te...,"[0.48427772521972656, 0.3821496367454529, 0.36..."
4,output_E3_typhoon_cosmetic_gpt4_shuffle_augmen...,"[0.5259838700294495, 0.33826255798339844, 0.49..."
5,output_E3_typhoon_cosmetic_gpt4_shuffle_augmen...,"[0.19092363119125366, 0.3748839795589447, 0.34..."
6,output_E3_typhoon_cosmetic_gpt4_shuffle_test.csv,"[0.3726959824562073, 0.5950114727020264, 0.397..."
7,output_E3_typhoon_cosmetic_gpt4_shuffle_test_a...,"[0.3779793977737427, 0.2787875533103943, 0.390..."
8,output_E3_typhoon_cosmetic_gpt4_test_augment.csv,"[0.3777066469192505, 0.38146883249282837, 0.61..."
9,output_E4_openthaigpt_cosmetic_gpt4_shuffle_au...,"[0.5227720141410828, 0.5343647599220276, 0.327..."


In [6]:
df = pd.read_csv(f"result.csv")
df

,mean_perplexity,filename,google_bleu,rouge1,rouge2,rougeL,bertscore_precision,bertscore_recall,bertscore_f1
0,0,output_E4_openthaigpt_cosmetic_gpt4_shuffle_te...,0.008453,0.001650,0.001083,0.001650,"[1.0000004768371582, 0.8208730816841125, 0.507...","[1.0000004768371582, 0.7791242599487305, 0.612...","[1.0000004768371582, 0.7994540333747864, 0.555..."
1,0,output_E3_typhoon_cosmetic_gpt4_shuffle_augmen...,0.003425,0.000000,0.000000,0.000000,"[1.0000004768371582, 1.0000004768371582, 0.762...","[1.0000004768371582, 1.0000004768371582, 0.771...","[1.0000004768371582, 1.0000004768371582, 0.766..."
2,0,output_E3_typhoon_cosmetic_gpt4_shuffle_test.csv,0.010637,0.000131,0.000101,0.000131,"[0.4570750594139099, 0.7678314447402954, 1.000...","[0.6488370299339294, 0.7691376209259033, 1.000...","[0.5363305807113647, 0.7684839963912964, 1.000..."
3,0,output_E0_wangchanglm_cosmetic_gpt4_test.csv,0.007282,0.000598,0.000418,0.000598,"[1.0000004768371582, 0.8273473978042603, 1.000...","[1.0000004768371582, 0.6926554441452026, 1.000...","[1.0000004768371582, 0.7540336847305298, 1.000..."
4,0,output_E4_openthaigpt_cosmetic_gpt4_shuffle_au...,0.001917,0.000697,0.000262,0.000697,"[1.0000004768371582, 0.4737371504306793, 0.796...","[1.0000004768371582, 0.6067236065864563, 0.766...","[1.0000004768371582, 0.5320461988449097, 0.781..."
5,0,output_E0_wangchanglm_cosmetic_noinput_gpt4_te...,0.000387,0.000000,0.000000,0.000000,"[0.5201453566551208, 0.6490975022315979, 0.420...","[0.7425912022590637, 0.6496220827102661, 0.758...","[0.6117750406265259, 0.6493597030639648, 0.540..."
6,0,output_E5_wangchanglm_furniture_gpt4_shuffle_a...,0.016267,0.000000,0.000000,0.000000,"[0.7152701616287231, 0.7236777544021606, 0.784...","[0.725098729133606, 0.719596266746521, 0.80093...","[0.720150887966156, 0.7216312885284424, 0.7927..."
7,0,output_E4_openthaigpt_cosmetic_gpt4_shuffle_au...,0.005933,0.002615,0.000872,0.002615,"[0.49851295351982117, 0.7927855253219604, 0.52...","[0.6305216550827026, 0.7862265706062317, 0.661...","[0.5567999482154846, 0.7894923686981201, 0.588..."
8,0,output_E4_wangchanglm_cosmetic_gpt4_shuffle_te...,0.002644,0.000872,0.000523,0.000872,"[0.48272472620010376, 1.0000004768371582, 0.71...","[0.7031645178794861, 1.0000004768371582, 0.691...","[0.5724563002586365, 1.0000004768371582, 0.701..."
9,0,output_E4_wangchanglm_cosmetic_gpt4_shuffle_au...,0.002548,0.000872,0.000523,0.000872,"[1.0000004768371582, 0.4451151192188263, 0.725...","[1.0000004768371582, 0.6367329359054565, 0.699...","[1.0000004768371582, 0.5239542722702026, 0.712..."


In [5]:
df =pd.read_csv("result_compare.csv")
#want to split _ in filename and move to new column
df['experiment']=df['filename'].apply(lambda x:x.split('_')[-1])
df.to_csv("result_compare.csv",index=False)


In [11]:
import os
import pandas as pd
import numpy as np
path_E="/home/natdanai/KeyToad/script/gentest/"
entries = os.listdir(path_E)
sorted_entries = sorted(entries)
print(len(sorted_entries))
list_keyword=["แปรงสีฟันผู้ใหญ่,ด้ามจับ,หัวกะทัดรัด,ขนแปรง,โค้งมน,อ่อนนุ่ม,เคลือบฟัน,เหงือก","แป้งผสมรองพื้น,เนื้อเนียนละเอียด,รูขุมขน,แสงแดด,เบาสบาย",
dataframe_list=[]
for filename in sorted_entries:
    df=pd.read_csv(path_E+filename)
    coun_list=[]
    for i in range(len(df)):
        list_word=df['word'][i].split(",")
        #count = sum(1 for keyword in keywords if keyword in text)
        count = sum(1 for word in list_word if word in df['response'][i])
        coun_list.append(count / len(list_word))

    dict_result={}
    dict_result['filename']=filename
    dict_result['count']=np.mean(coun_list)
    dataframe_list.append(dict_result)
df=pd.DataFrame(dataframe_list)
df.to_csv(f"result_count.csv",index=False)

31
